In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pandas import Series, DataFrame
import math

np.set_printoptions(suppress=True)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.5g}'.format

In [25]:
def Make_df_for_logistic(Item_name, co_val_list):
    '''
    slackで共有した各クラスごとの商品別CM視聴者人数のDataFrameを作成する関数です。引数は前共有したDataFrameに加え、zero_listからnot_zero_itemに加える商品のlist, およびmap_dict_nameを設定（いちいち大量のセルを実行するのが面倒だと思ったので関数化してます）
    '''
    df_tvcm = pd.read_csv("テレビCM出稿データ_2020.csv", encoding="cp932")
    df_item = pd.read_csv("item_yama.csv")#商品とアンケート1回目,2回目のデータがあればOK
    df_lca = pd.read_csv("fin_lca3.csv", index_col=0) #poLCAによる潜在クラスとSampleIDがあればどんな形のデータでもOK
    df_tv = pd.read_csv("テレビ番組別視聴状況_2020.csv")
    df_main = pd.read_csv("メインデータ_2020.csv")

    df_lca1 = df_lca.query('Class == 1')
    df_lca2 = df_lca.query('Class == 2')
    df_lca3 = df_lca.query('Class == 3')
    index1 = df_lca1.SampleID.unique()
    index2 = df_lca2.SampleID.unique()
    index3 = df_lca3.SampleID.unique()
    Class1 = df_tv.query("SampleID in @index1")
    Class2 = df_tv.query("SampleID in @index2")
    Class3 = df_tv.query("SampleID in @index3")
    df_item.drop(columns="num", inplace=True)

    add_list = []
    map_dic_name = {}

    if Item_name in map_dic_name.keys():
        Item_name = map_dic_name[Item_name]

    Covariate_list = co_val_list

    tv_day_text = df_tvcm["tv_day_text"].unique()
    map_dic = dict()

    for i in tv_day_text:
        key = i
        temp = i
        temp = temp.replace("/", "")
        if len(temp) == 6:
            temp = list(temp)
            temp.insert(5, "0")
        temp = ",".join(temp)
        temp = int(temp.replace(",",""))
        value = temp
        map_dic[key] = value

    df_tvcm["day"] = df_tvcm["tv_day_text"].map(map_dic)
    
    for i in ["day1", "day2"]:
        day_list = df_item[i].unique()
        map_dict_temp = dict()
        for j in day_list:
            key = j
            temp = j
            temp = temp.replace("/", "")
            if len(temp) == 6:
                temp = list(temp)
                temp.insert(5, "0")
            temp = ",".join(temp)
            temp = int(temp.replace(",",""))
            value = temp
            map_dict_temp[key] = value
        df_item[i] = df_item[i].map(map_dict_temp)

    item_list = df_item["item"].unique()
    not_zero_item = []
    not_zero_com = []
    zero_list = []
    for i in item_list:
        temp1 = df_tvcm[df_tvcm["item_name"] == i]
        temp2 = df_tvcm[df_tvcm["company_name"] == i]
        if len(temp1) != 0:
            not_zero_item.append(i)
        if len(temp2) != 0:
            not_zero_com.append(i)
        if len(temp1) == len(temp2) == 0:
            zero_list.append(i)
    
    for i in add_list:
        not_zero_item.append(i)
    
    for i in range(len(df_item)):
        if (df_item.iat[i, 0] in map_dic_name.keys())  == True:
            keys =  df_item.iat[i, 0]
            df_item.iat[i, 0] = map_dic_name[keys]

    df_Class = [Class1, Class2, Class3]
    INDEXs = [index1, index2, index3]
    return_list = []
    temp_column_list = Covariate_list
    temp_column_list.append("Watch")
    temp_column_list.append("SampleID")
    
    for Class, index in zip(df_Class, INDEXs):
        if (Item_name in not_zero_item) == True:
            temp = df_tvcm[df_tvcm["item_name"] == Item_name]
            day1 = df_item[df_item["item"] == Item_name].iat[0,1]
            day2 = df_item[df_item["item"] == Item_name].iat[0,2]
            select_df = temp.query("@day1 < day < @day2")
            temp_list = select_df["title_code_variable"].unique()

            temp_list2 = list()
            for m in temp_list:
                for n in Class["SampleID"][Class[m] == 1].unique():
                    temp_list2.append(n)
            SampleIDs = list(set(temp_list2))
        
        else:
            temp = df_tvcm[df_tvcm["company_name"] == Item_name]
            day1 = df_item[df_item["item"] == Item_name].iat[0,1]
            day2 = df_item[df_item["item"] == Item_name].iat[0,2]
            select_df = temp.query("@day1 < day < @day2")
            temp_list = select_df["title_code_variable"].unique()

            temp_list2 = list()
            for m in temp_list:
                for n in Class["SampleID"][Class[m] == 1].unique():
                    temp_list2.append(n)
            SampleIDs = list(set(temp_list2))
        
        main_Class = df_main.query("SampleID in @index")
        main_Class["Watch"] = 0
        for i in SampleIDs:
            main_Class["Watch"][main_Class["SampleID"] == i] = 1

        Class = main_Class.loc[:, temp_column_list]
        return_list.append(Class)
    
    return return_list


In [3]:
def Logistic(df_for_logistic, co_val_list):
    co_val_list_Log = co_val_list
    temp = co_val_list_Log
    df = df_for_logistic
    for col in df.columns:
        df[col] = df[col].astype(str)
    df["Watch"] = df["Watch"].astype(int)
    df.replace(" ", np.nan, inplace=True)
    df.dropna(inplace=True)

    y = list(["Watch", "~"])
    for i in temp:
        y.append(i)
        y.append("+")
    y.pop(-1)
    formula = " ".join(y)
    result = smf.glm(formula = formula, data = df, family = sm.families.Binomial(sm.families.links.logit)).fit()
    
    X = df.loc[:,co_val_list_Log]
    df["PS"] = result.predict(X)
    return_df = df.loc[:, ["SampleID", "Watch", "PS"]]
    return return_df

In [4]:
def DID_all(df_did):
    col = ["SampleID", "Value3", "Value4", "Watch", "PS"]
    df_did.columns = col
    df_did.replace(" ", np.nan, inplace=True)
    df_did.dropna(inplace=True)
    df_did["Value3"] = df_did["Value3"].astype(int)
    df_did["Value4"] = df_did["Value4"].astype(int)
    df_did["Watch"] = df_did["Watch"].astype(int)

    map_s = {1:2, 2:1, 3:3, 4:4, 5:5, 6:6}
    df_did["Value3"] = df_did["Value3"].map(map_s)
    df_did["Value4"] = df_did["Value4"].map(map_s)

    Pz_1 = np.average(df_did["Watch"])
    did_sum = 0
    for i in range(len(df_did)):
        data_i = df_did.iloc[i]
        data_i["PS"] = round(data_i["PS"], 6)
        if data_i["PS"] < 1.0:
            number = (data_i.loc["Value4"] - data_i.loc["Value3"])*(data_i.loc["Watch"] - data_i.loc["PS"])/(Pz_1*(1 - data_i.loc["PS"]))
            did_sum += number
    did_hat = did_sum/len(df_did)
    return did_hat

In [5]:
def DID_min_base(df_did):
    col = ["SampleID", "Value3", "Value4", "Watch", "PS"]
    df_did.columns = col
    df_did.replace(" ", np.nan, inplace=True)
    df_did.dropna(inplace=True)
    
    ##数値の変換
    min_base_map = {1:1, 2:1, 3:0, 4:0, 5:0, 6:0}
    df_did["Value3"] = df_did["Value3"].astype(int)
    df_did["Value4"] = df_did["Value4"].astype(int)
    min3 = df_did["Value3"].min()
    min4 = df_did["Value4"].min()
    if min3 != min4:
        print(f"min3 = {min3}")
        print(f"min4 = {min4}")
    else:
        print(f"min = {min3}")
    df_did["Value3"] = df_did["Value3"].map(min_base_map)
    df_did["Value4"] = df_did["Value4"].map(min_base_map)
    
    df_did["Watch"] = df_did["Watch"].astype(int)
    Pz_1 = np.average(df_did["Watch"])
    did_sum = 0
    for i in range(len(df_did)):
        data_i = df_did.iloc[i]
        data_i["PS"] = round(data_i["PS"], 6)
        if data_i["PS"] < 1.0:
            number = (data_i.loc["Value4"] - data_i.loc["Value3"])*(data_i.loc["Watch"] - data_i.loc["PS"])/(Pz_1*(1 - data_i.loc["PS"]))
            did_sum += number
    did_hat = did_sum/len(df_did)
    return did_hat

In [6]:
def DID_max_base(df_did):
    col = ["SampleID", "Value3", "Value4", "Watch", "PS"]
    df_did.columns = col
    df_did.replace(" ", np.nan, inplace=True)
    df_did.dropna(inplace=True)

    #数値の変換
    max_dic1 = {1:1, 2:1, 3:1, 4:1, 5:1, 6:0}
    max_dic2 = {1:1, 2:1, 3:1, 4:1, 5:0}
    max_dic3 = {1:1, 2:1, 3:1, 4:0}
    df_did["Value3"] = df_did["Value3"].astype(int)
    df_did["Value4"] = df_did["Value4"].astype(int)
    max3 = df_did["Value3"].max()
    max4 = df_did["Value4"].max()
    if max3 >= max4:
        if max3 == 6:
            df_did["Value3"] = df_did["Value3"].map(max_dic1)
            df_did["Value4"] = df_did["Value4"].map(max_dic1)
            print("base = 6")
        elif max3 == 5:
            df_did["Value3"] = df_did["Value3"].map(max_dic2)
            df_did["Value4"] = df_did["Value4"].map(max_dic2)
            print("base = 5")
        else:
            df_did["Value3"] = df_did["Value3"].map(max_dic3)
            df_did["Value4"] = df_did["Value4"].map(max_dic3)
            print("base = 4")    
    else:
        if max4 == 6:
            df_did["Value3"] = df_did["Value3"].map(max_dic1)
            df_did["Value4"] = df_did["Value4"].map(max_dic1)
            print("base = 6")
        elif max4 == 5:
            df_did["Value3"] = df_did["Value3"].map(max_dic2)
            df_did["Value4"] = df_did["Value4"].map(max_dic2)
            print("base = 5")
        else:
            df_did["Value3"] = df_did["Value3"].map(max_dic3)
            df_did["Value4"] = df_did["Value4"].map(max_dic3)
            print("base = 4")
    
    df_did["Watch"] = df_did["Watch"].astype(int)
    Pz_1 = np.average(df_did["Watch"])
    did_sum = 0
    for i in range(len(df_did)):
        data_i = df_did.iloc[i]
        data_i["PS"] = round(data_i["PS"], 6)
        if data_i["PS"] < 1.0:
            number = (data_i.loc["Value4"] - data_i.loc["Value3"])*(data_i.loc["Watch"] - data_i.loc["PS"])/(Pz_1*(1 - data_i.loc["PS"]))
            did_sum += number
    did_hat = did_sum/len(df_did)
    return did_hat

In [7]:
def all_did_all():
    main = pd.read_csv("メインデータ_2020.csv")
    results_dic = dict()
    Item = pd.read_csv("item.csv")
    Item = Item.query("num == 1")
    Item.drop(columns="num", inplace=True)
    item_list = Item["item"].unique()
    count=1


    for item in item_list:
        df_logs = Make_df_for_logistic(item, ["INCOM_SA", "JOB_CD", "FAMILY_CD"])
        temp_result_list = list()
        for df_class_i in df_logs:
            df_class_i = DataFrame(df_class_i)
            pre_df_did = Logistic(df_class_i, ["INCOM_SA", "JOB_CD", "FAMILY_CD"])
            temp_IDs = pre_df_did["SampleID"]
            Value3 = Item.query("item == @item")["Value3"]
            Value4 = Item.query("item == @item")["Value4"]
            li3 = main[Value3]
            li4 = main[Value4]
            temp = pd.concat([main.SampleID, li3, li4], axis = 1)
            temp = temp.query("SampleID == list(@temp_IDs)")
            pre_df_did.drop(columns="SampleID", inplace=True)
            df_did = pd.concat([temp, pre_df_did], axis=1)
            result = DID_all(df_did)
            temp_result_list.append(result)
            print(f"{count}終了")
            count += 1
        results_dic[item] = temp_result_list

    result_df = DataFrame(results_dic.values(), index=results_dic.keys(), 
    columns=["class1","class2", "class3"])
    return result_df

In [8]:
def all_did_min_base():
    main = pd.read_csv("メインデータ_2020.csv")
    results_dic = dict()
    Item = pd.read_csv("item.csv")
    Item = Item.query("num == 1")
    Item.drop(columns="num", inplace=True)
    item_list = Item["item"].unique()
    count=1


    for item in item_list:
        df_logs = Make_df_for_logistic(item, ["INCOM_SA", "JOB_CD", "FAMILY_CD"])
        temp_result_list = list()
        for df_class_i in df_logs:
            df_class_i = DataFrame(df_class_i)
            pre_df_did = Logistic(df_class_i, ["INCOM_SA", "JOB_CD", "FAMILY_CD"])
            temp_IDs = pre_df_did["SampleID"]
            Value3 = Item.query("item == @item")["Value3"]
            Value4 = Item.query("item == @item")["Value4"]
            li3 = main[Value3]
            li4 = main[Value4]
            temp = pd.concat([main.SampleID, li3, li4], axis = 1)
            temp = temp.query("SampleID == list(@temp_IDs)")
            pre_df_did.drop(columns="SampleID", inplace=True)
            df_did = pd.concat([temp, pre_df_did], axis=1)
            result = DID_min_base(df_did)
            temp_result_list.append(result)
            print(f"{count}終了")
            count += 1
        results_dic[item] = temp_result_list

    result_df = DataFrame(results_dic.values(), index=results_dic.keys(), 
    columns=["class1","class2", "class3"])
    return result_df

In [9]:
def all_did_max_base():
    main = pd.read_csv("メインデータ_2020.csv")
    results_dic = dict()
    Item = pd.read_csv("item.csv")
    Item = Item.query("num == 1")
    Item.drop(columns="num", inplace=True)
    item_list = Item["item"].unique()
    count=1


    for item in item_list:
        df_logs = Make_df_for_logistic(item, ["INCOM_SA", "JOB_CD", "FAMILY_CD"])
        temp_result_list = list()
        for df_class_i in df_logs:
            df_class_i = DataFrame(df_class_i)
            pre_df_did = Logistic(df_class_i, ["INCOM_SA", "JOB_CD", "FAMILY_CD"])
            temp_IDs = pre_df_did["SampleID"]
            Value3 = Item.query("item == @item")["Value3"]
            Value4 = Item.query("item == @item")["Value4"]
            li3 = main[Value3]
            li4 = main[Value4]
            temp = pd.concat([main.SampleID, li3, li4], axis = 1)
            temp = temp.query("SampleID == list(@temp_IDs)")
            pre_df_did.drop(columns="SampleID", inplace=True)
            df_did = pd.concat([temp, pre_df_did], axis=1)
            result = DID_max_base(df_did)
            temp_result_list.append(result)
            print(f"{count}終了")
            count += 1
        results_dic[item] = temp_result_list

    result_df = DataFrame(results_dic.values(), index=results_dic.keys(), 
    columns=["class1","class2", "class3"])
    return result_df

In [ ]:
result_all = -1*all_did_all()
result_all.to_csv("DID_PS_all.csv")

result_min_base = all_did_min_base()
result_min_base.to_csv("DID_PS_left.csv")

result_max_base = all_did_max_base()
result_max_base.to_csv("DID_PS_right.csv")